ဒီ Project ရဲ့ ရည်ရွယ်ချက်က လျှောက်ထားသူရဲ့ ဝင်ငွေ၊ ပညာအရည်အချင်း၊ အလုပ်အကိုင်နဲ့ Credit Score တွေကို ကြည့်ပြီး "ချေးငွေ ထုတ်ပေးသင့် (Approved)" သို့မဟုတ် "ငြင်းပယ်သင့် (Rejected)" ဆိုတာကို AI သုံးပြီး ဆုံးဖြတ်မှာ ဖြစ်ပါတယ်။

အဆင့် (၁) - လိုအပ်တဲ့ Tools (Libraries) များ ခေါ်ယူခြင်း
ပထမဆုံး data တွေကို ကိုင်တွယ်ဖို့နဲ့ AI model ဆောက်ဖို့ လိုအပ်တဲ့ library တွေကို ခေါ်ရပါမယ်။

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Warning တွေမတက်အောင် ပိတ်ထားခြင်း
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'pandas'

အဆင့် (၂) - Data ကို ဖတ်ခြင်း (Loading Data)
Kaggle မှာရှိတဲ့ loan dataset ကို ဆွဲသွင်းပါမယ်။ (Dataset path က loan_approval_dataset.csv ဖြစ်တယ်လို့ ယူဆပါမယ်)

In [2]:
# Data ဖတ်မယ်
df = pd.read_csv('loan_approval_dataset.csv')

# Data ဘယ်နှကြောင်းပါလဲ စစ်မယ်
print("Data Shape:", df.shape)

# ရှေ့ဆုံး ၅ ကြောင်းကို ကြည့်မယ်
display(df.head())

Data Shape: (4269, 13)


,loan_id,no_of_dependents,education,self_employed,income_annum,loan_amount,loan_term,cibil_score,residential_assets_value,commercial_assets_value,luxury_assets_value,bank_asset_value,loan_status
0,1,2,Graduate,No,9600000,29900000,12,778,2400000,17600000,22700000,8000000,Approved
1,2,0,Not Graduate,Yes,4100000,12200000,8,417,2700000,2200000,8800000,3300000,Rejected
2,3,3,Graduate,No,9100000,29700000,20,506,7100000,4500000,33300000,12800000,Rejected
3,4,3,Graduate,No,8200000,30700000,8,467,18200000,3300000,23300000,7900000,Rejected
4,5,5,Not Graduate,Yes,9800000,24200000,20,382,12400000,8200000,29400000,5000000,Rejected


အဆင့် (၃) - Data ပြင်ဆင်ခြင်း (Preprocessing)
Machine Learning model တွေက စာသား (Text) တွေကို တိုက်ရိုက် နားမလည်ပါဘူး။ ဒါကြောင့် စာသားတွေကို ကိန်းဂဏန်း ပြောင်းပေးရပါမယ်။

In [ ]:
# ၁။ Column နာမည်တွေမှာ ပါနေတဲ့ space တွေကို အရင်ဖယ်မယ်
df.columns = df.columns.str.strip()

# ၂။ Value တွေထဲမှာ ပါနေတဲ့ space တွေကိုပါ ဖယ်မယ် (ဒါမှ mapping လုပ်ရတာ အဆင်ပြေမှာပါ)
for col in ['education', 'self_employed', 'loan_status']:
    df[col] = df[col].str.strip()

# ၃။ စာသားတွေကို နံပါတ်ပြောင်းမယ်
df['education'] = df['education'].map({'Graduate': 1, 'Not Graduate': 0})
df['self_employed'] = df['self_employed'].map({'Yes': 1, 'No': 0})
df['loan_status'] = df['loan_status'].map({'Approved': 1, 'Rejected': 0})

# ၄။ မလိုအပ်တဲ့ loan_id ကို ဖြုတ်မယ်
X = df.drop(columns=['loan_id', 'loan_status'])
y = df['loan_status']

print("ပြင်ဆင်ပြီးသော Column များ:", X.columns.tolist())
display(X.head())

ပြင်ဆင်ပြီးသော Features များ -  ['no_of_dependents', 'education', 'self_employed', 'income_annum', 'loan_amount', 'loan_term', 'cibil_score', 'residential_assets_value', 'commercial_assets_value', 'luxury_assets_value', 'bank_asset_value']


,no_of_dependents,education,self_employed,income_annum,loan_amount,loan_term,cibil_score,residential_assets_value,commercial_assets_value,luxury_assets_value,bank_asset_value
0,2,NaN,NaN,9600000,29900000,12,778,2400000,17600000,22700000,8000000
1,0,NaN,NaN,4100000,12200000,8,417,2700000,2200000,8800000,3300000
2,3,NaN,NaN,9100000,29700000,20,506,7100000,4500000,33300000,12800000
3,3,NaN,NaN,8200000,30700000,8,467,18200000,3300000,23300000,7900000
4,5,NaN,NaN,9800000,24200000,20,382,12400000,8200000,29400000,5000000


အဆင့် (၄) - Model သင်ကြားခြင်း (Training)
ဒီအဆင့်မှာ train_test_split ကို သုံးပြီး data ခွဲပါမယ်။

In [ ]:
# Data ကို ၈၀% (သင်ကြားဖို့) နဲ့ ၂၀% (စစ်ဆေးဖို့) ခွဲမယ်
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Decision Tree Model ဆောက်မယ်
model = DecisionTreeClassifier(max_depth=5, random_state=42)
model.fit(X_train, y_train)

print("Training ပြီးပါပြီ။")

အဆင့် (၅) - ရလဒ်စစ်ဆေးခြင်း
Model က ဘယ်လောက် မှန်သလဲဆိုတာ ကြည့်ပါမယ်။

In [ ]:
y_pred = model.predict(X_test)
print(f"Accuracy Score: {accuracy_score(y_test, y_pred) * 100:.2f}%")

# Confusion Matrix ဆွဲကြည့်မယ်
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

အဆင့် (၆) - ရလဒ်ကို စစ်ဆေးခြင်း (Evaluation)
Model က ဘယ်လောက် မှန်ကန်အောင် ခန့်မှန်းနိုင်လဲဆိုတာ စစ်ဆေးပါမယ်။

In [ ]:
# Test data ကို သုံးပြီး ခန့်မှန်းခိုင်းမယ်
y_pred = model.predict(X_test)

# Accuracy (တိကျမှု) ကို ကြည့်မယ်
accuracy = accuracy_score(y_test, y_pred)
print(f"တိကျမှုနှုန်း (Accuracy): {accuracy * 100:.2f}%")

# Confusion Matrix ဆွဲမယ် (ဘယ်နှခု မှားလဲ ကြည့်ဖို့)
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Loan Approval Confusion Matrix')
plt.show()

အဆင့် (၇) - ဘယ်အချက်တွေက အရေးကြီးဆုံးလဲ? (Feature Importance)
ချေးငွေ ချပေးဖို့ ဆုံးဖြတ်တဲ့နေရာမှာ ဘယ်အချက်က အလွှမ်းမိုးဆုံးလဲဆိုတာ ကြည့်ပါမယ်။

In [ ]:
importance = pd.DataFrame({
    'feature': X.columns,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(10, 5))
sns.barplot(x='importance', y='feature', data=importance)
plt.title('Top Factors for Loan Approval')
plt.show()